In [ ]:
import argparse
import cv2
import os
from os.path import exists,join, split, dirname
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import sys 
sys.path.append('../')
from PIL import Image
import torch 
import torch.utils.data
from torch import nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from segment import SegList

import dla_up 
import data_transforms as transforms
import dataset

try:
    from modules import batchnormsync
    HAS_BN_SYNC = True 
except ImportError:
    HAS_BN_SYNC = True 
    
torch.set_default_tensor_type('torch.cuda.FloatTensor')

%matplotlib inline

In [ ]:
# import pytorch model 
# model = torch.load('../model_best.pth.tar')
# model = torch.load('../model_best.pth.tar')
model = torch.load('../checkpoint_1200.pth.tar')

In [ ]:
# idx = 'val/frankfurt/frankfurt_000000_000294_leftImg8bit'
idx = '2008_000033'
dir_prefix = 'c:/Users/biosim/oladapo/data/benchmark_RELEASE/dataset/img/'
# dir_prefix  = 'C:/Users/biosim/oladapo/data/Cityscape/leftImg8bit/'
if exists(dir_prefix):
    image = Image.open(dir_prefix+ idx + '.jpg')
    plt.imshow(np.asarray(image))
else:
    print("dir_prefix does not exist")

In [ ]:
phase = 'val'
data_dir = 'C:/Users/biosim/oladapo/data/benchmark_RELEASE/dataset/'
# data_dir = 'C:/Users/biosim/oladapo/data/Cityscape/'
t = []

info = dataset.load_dataset_info(data_dir)
normalize = transforms.Normalize(mean=info.mean, std=info.std)
# scales = [0.5, 0.75, 1.25, 1.5, 1.75]
scales = [0.5, 0.75, 1.25, 1.5]
t = []
crop_size = 416
# if args.crop_size > 0:
t.append(transforms.RandomCrop(crop_size))
# t.append(transforms.PadToSize(crop_size))
t.extend([transforms.ToTensor(), normalize])

data = SegList(data_dir, phase, transforms.Compose(t),
                       out_name=True, out_size=True,
                       binary= True)
test_loader = torch.utils.data.DataLoader(data,
batch_size=4, shuffle=False, num_workers=1,
pin_memory=False
)

In [ ]:
# j = 0
# for iter, (im, label, name, size) in enumerate(test_loader):
#     j = j+1
    
    
# #     tmp = np.reshape(tmp,(crop_size,crop_size, 3))
# #     plt.imshow(np.squeeze(im[0,:,:,:]))
#     plt.imshow(tmp)
#     plt.show()
    
#     if j > 0:
#         break

In [ ]:
j = 0
# print(model.keys())
single_model = dla_up.__dict__.get(model['arch'])(
        2, down_ratio=2)
# single_model.__dict__ = model
# print(single_model)
# print(model[model.keys()[0]])


new_model = torch.nn.DataParallel(single_model).cuda()
# print(new_model)

# checkpoint = torch.load('../')

# start_epoch = checkpoint['epoch']

# best_prec1 = checkpoint['best_prec1']


new_model.load_state_dict(model['state_dict'])


# new_model = torch.nn.DataParallel(single_model).cuda()
new_model.eval()
# new_model.__dict__ = model
# print(new_model.module)
# # new_model.load_state_dict(model['state_dict'])
k =2
boundaries_prefix = 'C:/Users/biosim/oladapo/data/benchmark_RELEASE/dataset/boundaries/'
for  iter, (im, label, name, size) in enumerate(test_loader):
    image_var = Variable(im, requires_grad=False, volatile=True)
    image_var = image_var.cuda()
    print(name)
    name_idx = name[k]
    gt_filename = boundaries_prefix + phase +'/' + name_idx[4:-3] + 'png'
    gt_image = plt.imread(gt_filename)
    print(gt_filename)
    final = new_model.module(image_var)[0]
#     final = single_model(image_var)[0]
    _, pred = torch.max(final, 1)
    pred = pred.cpu().data.numpy()
    print(pred.shape)
    print(np.amax(pred[k,:,:]))
    print(np.amin(pred[k,:,:]))
    print(pred[k,:,:])
    pred_im = []
    pred_im = Image.fromarray(pred[k].astype(np.uint8))
    input_im =np.squeeze(im.numpy()[k])
    tmp = np.rollaxis(input_im, 0, 3)
    print(tmp.shape)
    plt.imshow(tmp)
    plt.show()
    plt.imshow(np.asarray(pred_im)*255)
    plt.show()
    plt.imshow(gt_image*255)
    plt.show()
    print(np.asarray(pred_im))
    j = j+1
    if j>0:
        break